<a href="https://colab.research.google.com/github/amanda-araujo/otimizacao-inteira/blob/main/AC2_OtimizacaoInteira.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**SME0213 Otimização Inteira - ICMC-USP**

Amanda Araujo Silva - 10260441

In [24]:
%pip install gurobipy

In [25]:
import gurobipy as gp

# Atividade Complementar 2: Corte de jumbo



## Problema de corte

Problema clássico de otimização: problema de corte.

$\quad$ Dados do problema:

*    Comprimento bobina jumbo $L$
*    Comprimentos dos itens $l_i$: comprimento item $i$
*    Demanda de cada item $d_i$: demanda item $i$

$\quad$Modelagem geral (Kantorovich):

> $Min \sum_{k = 1}^K y_k$

> $S.a. \sum_{k = 1}^K \alpha_{jk} \ge d_j \quad j = 1, ..., n$

> $l_1\alpha_{1k} + l_2\alpha_{2k} + ... + l_n\alpha_{nk} \le L y_k \quad k = 1, ..., K$

> $\alpha_{jk} \ge 0$ e inteiro $\quad j = 1, ..., n \quad k = 1, ..., K$

> $y_k \in \{0, 1\} \quad k = 1, ..., K$

sendo:
*    $y_k$: variável de decisão binária; 1 se a barra $k$ é cortada, 0 c.c.;
*    $\alpha_{jk}$: variável de decisão inteira; número de vezes que o item $j$ é cortado na barra $k$;
*    $K$: valor limite para o número de barras (estimativa);
*    $n$: número de itens.

## Implementação Kantorovich

In [33]:
from gurobipy import Model, GRB
import math

# Criação do Modelo
model = Model("jumbo")

# Dados do problema
L = 400.0 # tamanho bobina jumbo
l = [40.0, 45.0, 55.0, 60.0] # tamanho itens
d = [12.0, 20.0, 42.0, 18.0] # demanda itens

## Parâmetros
n = 4   # número de itens
K = (int(math.ceil( sum(l[i] * d[i] for i in range(n))/L ))) # valor limite nº de barras; menor limiar
print("Tamanho bobina jumbo:", L)
print("Demanda total:", sum(l[i] * d[i] for i in range(n)))
print("Valor limite K: ", K)
print("\n")

# Adição de variáveis
alpha = model.addVars(n, K, vtype=GRB.INTEGER, name="alpha", lb=0) # inteira não negativa Z+
y = model.addVars(K, vtype=GRB.BINARY, name="y")                   # binária

# Definição função objetivo: minimizar número de barras utilizadas
model.setObjective(
    sum(y[k] for k in range(K)),
    GRB.MINIMIZE
)

# Restrições
## Demanda
for j in range(n):
  model.addConstr(sum(alpha[j, k] for k in range(K)) >= d[j])

## Tamanho bobina jumbo
for k in range(K):
  model.addConstr(sum(l[i] * alpha[i, k] for i in range(n)) <= L * y[k])

## Restrição alpha >= 0 já add pelo uso de lb (lower bound) ao definir alpha
## Restrição y binário já add ao definir y

# Otimização
model.optimize()


# Escrita do modelo em formato LP
model.write('jumbo.lp')

# Salva a solução do problema
model.write('jumbo.sol')

# Exibe os resultados
if model.status == GRB.OPTIMAL:
    for i in range(n):
      for k in range(K):
        if alpha[i, k].X > 0:
          print(f"alpha[{i}, {k}] = {alpha[i, k].X}")

    for k in range(K):
      if y[k].X > 0:
        print(f"y[{k}] = {y[k].X}")

    print(f"z = {model.ObjVal}")

else:
    print("Solução ótima não encontrada.")

Tamanho bobina jumbo: 400.0
Demanda total: 4770.0
Valor limite K:  12


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 16 rows, 60 columns and 108 nonzeros
Model fingerprint: 0x99dd66f5
Variable types: 0 continuous, 60 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 4e+01]
Presolve time: 0.00s
Presolved: 16 rows, 60 columns, 108 nonzeros
Variable types: 0 continuous, 60 integer (12 binary)

Root relaxation: objective 1.192500e+01, 45 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   11.92500    0    9          -   11

## Padrões de corte encontrados, usos e perdas


Cada padrão de corte representa como uma bobina jumbo é utilizada, ou seja, quantas unidades de cada item são cortadas dela: representado pelas variáveis alpha[i, k].

*    $\alpha_{jk}$: número de vezes que o item $j$ é cortado na barra $k$

### Padrão de corte por bobina

In [27]:
# Padrões de corte encontrados (por barra)

print("\n" + "=" * 45)
print("Padrões de corte encontrados (por bobina):")
print("=" * 45)

## Cabeçalho com os comprimentos dos itens
header = "Bobina | " + " | ".join([f"{int(li):>4}" for li in l])
print(header)
print("-" * (len(header) + 3))

## Para cada bobina usada, mostrar a linha com os cortes
for k in range(K):
    if y[k].X > 0.5:
        linha = f"{k:>6} |"
        for i in range(n):
            qtd = int(alpha[i, k].X + 0.5)
            cell = f"{qtd:>4}" if qtd > 0 else "    "
            linha += f" {cell} |"
        print(linha)

print("=" * 45)
print(f"Total de bobinas utilizadas: {int(model.ObjVal)}")
print("=" * 45)


Padrões de corte encontrados (por bobina):
Bobina |   40 |   45 |   55 |   60
-------------------------------------
     0 |      |    5 |    1 |    2 |
     1 |    1 |    3 |    4 |      |
     2 |      |      |    5 |    2 |
     3 |      |      |    4 |    3 |
     4 |      |    4 |    4 |      |
     5 |    3 |      |    4 |    1 |
     6 |    1 |    3 |    3 |    1 |
     7 |      |      |    5 |    2 |
     8 |      |    1 |    2 |    4 |
     9 |    3 |      |    5 |      |
    10 |    3 |      |    5 |      |
    11 |    1 |    4 |      |    3 |
Total de bobinas utilizadas: 12


### Padrões de corte únicos, usos e perdas

In [28]:
# Padrões de corte únicos encontrados, usos e perdas
from collections import defaultdict

# Contar padrões de corte e calcular perdas
padroes_unicos = defaultdict(int)
padroes_perda = {}

for k in range(K):
    if y[k].X > 0.5:
        padrao = tuple(int(alpha[i, k].X + 0.5) for i in range(n))
        padroes_unicos[padrao] += 1
        if padrao not in padroes_perda:
            usado = sum(padrao[i] * l[i] for i in range(n))
            perda = L - usado
            padroes_perda[padrao] = perda

print(f"Total de bobinas utilizadas: {int(model.ObjVal)}")
print(f"\nNúmero de padrões de corte distintos: {len(padroes_unicos)}")

# Exibir a tabela final
print("\n" + "=" * 45)
print(f"{'Padrão de corte':<20} | {'Usos':^5} | {'Perda':^6}")
print("-" * 45)

for padrao in padroes_unicos:
    usos = padroes_unicos[padrao]
    perda = padroes_perda[padrao]
    print(f"{str(padrao):<20} | {usos:^5} | {perda:^6.1f}")

print("=" * 45)

# Perda total
perda_total = sum(padroes_unicos[p] * padroes_perda[p] for p in padroes_unicos)
print(f"Perda total acumulada: {perda_total:.1f}")
print("=" * 45)


Total de bobinas utilizadas: 12

Número de padrões de corte distintos: 10

Padrão de corte      | Usos  | Perda 
---------------------------------------------
(0, 5, 1, 2)         |   1   |  0.0  
(1, 3, 4, 0)         |   1   |  5.0  
(0, 0, 5, 2)         |   2   |  5.0  
(0, 0, 4, 3)         |   1   |  0.0  
(0, 4, 4, 0)         |   1   |  0.0  
(3, 0, 4, 1)         |   1   |  0.0  
(1, 3, 3, 1)         |   1   |  0.0  
(0, 1, 2, 4)         |   1   |  5.0  
(3, 0, 5, 0)         |   2   |  5.0  
(1, 4, 0, 3)         |   1   |  0.0  
Perda total acumulada: 30.0


## Exploração valor de K

Ao se alterar o valor de K, obtivemos o seguinte comportamento:

*   K < 12: Model is infeasible
*   K = 12: retorna solução ótima, 10 padrões de corte distintos
*   K = 15: retorna solução ótima, mesmo número de bobina e mesma perda total, no entanto, nº distinto de padrões de corte (6)    

> Será que há múltiplas soluções ótimas? I.e. diferentes combinações de padrões de corte distintas que minimizam o número de bobinas jumbo a serem cortadas?

Pode haver outras soluções de valor 12 que o solver não listou (por padrão o Gurobi só guarda algumas poucas soluções) - chamados ótimos alternativos.

Verificação de múltiplas soluções ótimas (parâmetros Gurobi - pedir explicitamente para explorar a *solution pool*):

*    PoolSearchMode=2 → explora o conjunto de soluções ótimas (em vez de parar no primeiro ótimo).
*    PoolSolutions → define quantas soluções distintas o solver deve guardar.


In [29]:
# Verificação de múltiplas soluções ótimas
from gurobipy import Model, GRB
import math

# Criação do Modelo
model = Model("jumbo")

# Dados do problema
L = 400.0 # tamanho bobina jumbo
l = [40.0, 45.0, 55.0, 60.0] # tamanho itens
d = [12.0, 20.0, 42.0, 18.0] # demanda itens

## Parâmetros
n = 4   # número de itens
K = 12 #(int(math.ceil( sum(l[i] * d[i] for i in range(n))/L ))) # valor limite nº de barras; menor limiar
print("Tamanho bobina jumbo:", L)
print("Demanda total:", sum(l[i] * d[i] for i in range(n)))
print("Valor limite K: ", K)
print("\n")

# Adição de variáveis
alpha = model.addVars(n, K, vtype=GRB.INTEGER, name="alpha", lb=0) # inteira não negativa Z+
y = model.addVars(K, vtype=GRB.BINARY, name="y")                   # binária

# Definição função objetivo: minimizar número de barras utilizadas
model.setObjective(
    sum(y[k] for k in range(K)),
    GRB.MINIMIZE
)

# Restrições
## Demanda
for j in range(n):
  model.addConstr(sum(alpha[j, k] for k in range(K)) >= d[j])

## Tamanho bobina jumbo
for k in range(K):
  model.addConstr(sum(l[i] * alpha[i, k] for i in range(n)) <= L * y[k])

## Restrição alpha >= 0 já add pelo uso de lb (lower bound) ao definir alpha
## Restrição y binário já add ao definir y

# Configuração do Solution Pool
model.Params.PoolSearchMode = 2   # Explorar todo o pool de soluções ótimas
model.Params.PoolSolutions = 1000 # Número máximo de soluções a guardar
model.Params.PoolGap = 0.0       # Apenas soluções realmente ótimas (não quase-ótimas)

# Otimização
model.optimize()


# Escrita do modelo em formato LP
model.write('jumbo.lp')

# Salva a solução do problema
model.write('jumbo.sol')

# Exibe os resultados
if model.status == GRB.OPTIMAL:
    print(f"\nNúmero de soluções encontradas: {model.SolCount}")
    print(f"Valor ótimo da função objetivo: {model.ObjVal}\n")

    # Itera sobre todas as soluções ótimas
    for sol in range(model.SolCount):
        model.Params.SolutionNumber = sol
        print(f"=== Solução {sol+1} ===")
        for i in range(n):
            for k in range(K):
                if alpha[i, k].Xn > 1e-6:  # usa Xn para acessar solução específica
                    print(f"alpha[{i}, {k}] = {alpha[i, k].Xn:.0f}")
        for k in range(K):
            if y[k].Xn > 0.5:
                print(f"y[{k}] = {y[k].Xn:.0f}")
        print(f"Objetivo = {model.PoolObjVal}\n")

else:
    print("Solução ótima não encontrada.")


A saída de streaming foi truncada nas últimas 5000 linhas.
y[0] = 1
y[1] = 1
y[2] = 1
y[3] = 1
y[4] = 1
y[5] = 1
y[6] = 1
y[7] = 1
y[8] = 1
y[9] = 1
y[10] = 1
y[11] = 1
Objetivo = 12.0

=== Solução 896 ===
alpha[0, 2] = 1
alpha[0, 3] = 2
alpha[0, 4] = 2
alpha[0, 5] = 1
alpha[0, 6] = 3
alpha[0, 8] = 2
alpha[0, 10] = 1
alpha[1, 0] = 1
alpha[1, 1] = 4
alpha[1, 2] = 3
alpha[1, 3] = 1
alpha[1, 4] = 2
alpha[1, 7] = 1
alpha[1, 8] = 1
alpha[1, 9] = 4
alpha[1, 10] = 3
alpha[2, 0] = 1
alpha[2, 1] = 4
alpha[2, 2] = 3
alpha[2, 3] = 5
alpha[2, 4] = 4
alpha[2, 6] = 4
alpha[2, 7] = 3
alpha[2, 8] = 5
alpha[2, 9] = 4
alpha[2, 10] = 3
alpha[2, 11] = 6
alpha[3, 0] = 5
alpha[3, 2] = 1
alpha[3, 5] = 6
alpha[3, 6] = 1
alpha[3, 7] = 3
alpha[3, 10] = 1
alpha[3, 11] = 1
y[0] = 1
y[1] = 1
y[2] = 1
y[3] = 1
y[4] = 1
y[5] = 1
y[6] = 1
y[7] = 1
y[8] = 1
y[9] = 1
y[10] = 1
y[11] = 1
Objetivo = 12.0

=== Solução 897 ===
alpha[0, 2] = 1
alpha[0, 3] = 2
alpha[0, 4] = 2
alpha[0, 5] = 1
alpha[0, 6] = 3
alpha[0, 8] = 2
a

Para K = 15, Gurobi foi capaz de encontrar 1000 soluções ótimas diferentes, visualmente distintas, mas será que não entra em jogo a questão da simetria das bobinas que faz com que o número de soluções ótimas possíveis aumente?

> Rodando exploração da solution pool para K = 12 também levou a pelo menos 1000 soluções ótimas, aparentemente distintas entre si. Sendo 12 o número inteiro mínimo estimado pela demanda para o número de barras, a alta quantidade de soluções listadas leva a crer que realmente há várias combinações de padrões de corte realmente distintas (não somente trocando o label da barra).

Foram encontradas pelo menos 1000 soluções ótimas em relação à minimização do número de bobinas. Mas será que a perda total também se mantém?

>> Objetivo principal do modelo é **minimizar o número de bobinas usadas**, mas entre soluções com o mesmo número de bobinas, algumas podem ser mais *eficientes* que outras em termos de perda de material.

Como escolher entre as múltiplas soluções ótimas encontradas pelo modelo?

No geral, queremos ou temos como prioridade:

1.   Número mínimo de bobinas (objetivo do modelo);
2.   Menor perda total (eficiência);
3.   Menor número de padrões distintos (facilidade operacional).

*    Decisão: ordenação das soluções pelos critérios 1, 2, 3.

In [30]:
# Exploração perda total & quantidade de padrões de corte distintos

if model.status == GRB.OPTIMAL:
    print(f"\nNúmero de soluções encontradas: {model.SolCount}")
    print(f"Valor ótimo da função objetivo: {model.ObjVal}\n")

    for sol in range(model.SolCount):
        model.Params.SolutionNumber = sol
        print(f"=== Solução {sol+1} ===")
        used_bars = 0
        total_loss = 0.0

        for k in range(K):
            bar_used = y[k].Xn > 0.5
            if bar_used:
                cut_length = sum(alpha[i,k].Xn * l[i] for i in range(n))
                loss = L - cut_length
                total_loss += loss
                used_bars += 1

        print(f"Número de padrões de corte distintos: {used_bars}")
        print(f"Perda total acumulada: {total_loss:.2f}")

        for i in range(n):
            for k in range(K):
                if alpha[i,k].Xn > 1e-6:
                    print(f"alpha[{i}, {k}] = {alpha[i,k].Xn:.0f}")
        for k in range(K):
            if y[k].Xn > 0.5:
                print(f"y[{k}] = {y[k].Xn:.0f}")
        print(f"Objetivo = {model.PoolObjVal}\n")

else:
    print("Solução ótima não encontrada.")

A saída de streaming foi truncada nas últimas 5000 linhas.
=== Solução 900 ===
Número de padrões de corte distintos: 12
Perda total acumulada: 30.00
alpha[0, 2] = 1
alpha[0, 3] = 2
alpha[0, 4] = 2
alpha[0, 5] = 1
alpha[0, 8] = 3
alpha[0, 11] = 3
alpha[1, 0] = 4
alpha[1, 1] = 4
alpha[1, 2] = 3
alpha[1, 3] = 1
alpha[1, 4] = 2
alpha[1, 7] = 1
alpha[1, 10] = 5
alpha[2, 0] = 4
alpha[2, 1] = 4
alpha[2, 2] = 3
alpha[2, 3] = 5
alpha[2, 4] = 4
alpha[2, 6] = 4
alpha[2, 7] = 3
alpha[2, 8] = 5
alpha[2, 9] = 4
alpha[2, 10] = 1
alpha[2, 11] = 5
alpha[3, 2] = 1
alpha[3, 5] = 6
alpha[3, 6] = 3
alpha[3, 7] = 3
alpha[3, 9] = 3
alpha[3, 10] = 2
y[0] = 1
y[1] = 1
y[2] = 1
y[3] = 1
y[4] = 1
y[5] = 1
y[6] = 1
y[7] = 1
y[8] = 1
y[9] = 1
y[10] = 1
y[11] = 1
Objetivo = 12.0

=== Solução 901 ===
Número de padrões de corte distintos: 12
Perda total acumulada: 30.00
alpha[0, 2] = 1
alpha[0, 3] = 2
alpha[0, 4] = 2
alpha[0, 5] = 1
alpha[0, 8] = 3
alpha[0, 11] = 3
alpha[1, 1] = 4
alpha[1, 2] = 3
alpha[1, 3] = 1
alph

Pela visualização do resultado truncado para as soluções ótimas encontradas de número 900 a 1000, todas elas apresentam:
1. Número de barras ótimo (12);
2. Mesma perda total acumulada (30.0);
3. Número de padrões de corte distintos igual a 12.

Note que: para o mesmo problema já vimos solução ótima exibida para K = 15 com o número de bobinas ótimo, perda 30.0, porém empregando apenas 6 padrões de corte distintos. Além disso, a primeira solução ótima encontrada de todas para K = 12 resultou em apenas 10 padrões de corte distintos. Ou seja, é posível realizar uma escolha priorizando a facilidade operacional.

In [31]:
# Ordenação das soluções

if model.status == GRB.OPTIMAL:
    solutions = []

    for sol in range(model.SolCount):
        model.Params.SolutionNumber = sol
        used_bars = 0
        total_loss = 0.0

        # Calcula perda total e número de padrões de corte
        for k in range(K):
            bar_used = y[k].Xn > 0.5
            if bar_used:
                cut_length = sum(alpha[i,k].Xn * l[i] for i in range(n))
                loss = L - cut_length
                total_loss += loss
                used_bars += 1

        # Salva a solução em um dicionário
        solutions.append({
            "sol_index": sol,
            "objective": model.PoolObjVal,
            "total_loss": total_loss,
            "used_bars": used_bars,
            "alpha": [[alpha[i,k].Xn for k in range(K)] for i in range(n)],
            "y": [y[k].Xn for k in range(K)]
        })

    # Ordena soluções: primeiro pela perda total, depois pelo número de padrões distintos
    solutions.sort(key=lambda s: (s["total_loss"], s["used_bars"]))

    # Exibe soluções ordenadas
    for idx, sol in enumerate(solutions):
        print(f"=== Solução {idx+1} (Pool index {sol['sol_index']}) ===")
        print(f"Objetivo (nº de bobinas): {sol['objective']}")
        print(f"Número de padrões de corte distintos: {sol['used_bars']}")
        print(f"Perda total acumulada: {sol['total_loss']:.2f}\n")

        for i in range(n):
            for k in range(K):
                if sol["alpha"][i][k] > 1e-6:
                    print(f"alpha[{i},{k}] = {sol['alpha'][i][k]:.0f}")
        for k in range(K):
            if sol["y"][k] > 0.5:
                print(f"y[{k}] = {sol['y'][k]:.0f}")
        print(f"---------------------------\n")

else:
    print("Solução ótima não encontrada.")

=== Solução 1 (Pool index 670) ===
Objetivo (nº de bobinas): 12.0
Número de padrões de corte distintos: 12
Perda total acumulada: 30.00

alpha[0,2] = 4
alpha[0,4] = 3
alpha[0,5] = 1
alpha[0,6] = 1
alpha[0,11] = 3
alpha[1,3] = 4
alpha[1,5] = 8
alpha[1,6] = 3
alpha[1,8] = 1
alpha[1,9] = 4
alpha[2,0] = 4
alpha[2,1] = 4
alpha[2,2] = 4
alpha[2,3] = 4
alpha[2,4] = 5
alpha[2,6] = 3
alpha[2,7] = 4
alpha[2,8] = 1
alpha[2,9] = 4
alpha[2,10] = 4
alpha[2,11] = 5
alpha[3,0] = 3
alpha[3,1] = 3
alpha[3,6] = 1
alpha[3,7] = 3
alpha[3,8] = 5
alpha[3,10] = 3
y[0] = 1
y[1] = 1
y[2] = 1
y[3] = 1
y[4] = 1
y[5] = 1
y[6] = 1
y[7] = 1
y[8] = 1
y[9] = 1
y[10] = 1
y[11] = 1
---------------------------

=== Solução 2 (Pool index 585) ===
Objetivo (nº de bobinas): 12.0
Número de padrões de corte distintos: 12
Perda total acumulada: 30.00

alpha[0,2] = 4
alpha[0,4] = 3
alpha[0,5] = 1
alpha[0,6] = 1
alpha[0,11] = 3
alpha[1,3] = 4
alpha[1,5] = 8
alpha[1,8] = 4
alpha[1,10] = 4
alpha[2,0] = 4
alpha[2,1] = 4
alpha[2,2] 

>> Essa ordenação (código adaptado pelo ChatGPT - preguiça de olhar para as saídas) não parece ter resultado em nada relevante em termos de se tomar uma decisão. Dentre as 1000 soluções ótimas encontradas, as soluções ótimas vistas anteriormente com apenas 10 e 6 padrões de corte diferentes não apareceram para emcabeçar a lista.

> Vamos parar por aqui. Por hoje é só! Valeu o aprendizado, *That's all folks!*

### Apêndice: Resultado K = 15

In [34]:
from gurobipy import Model, GRB
import math

# Criação do Modelo
model = Model("jumbo")

# Dados do problema
L = 400.0 # tamanho bobina jumbo
l = [40.0, 45.0, 55.0, 60.0] # tamanho itens
d = [12.0, 20.0, 42.0, 18.0] # demanda itens

## Parâmetros
n = 4   # número de itens
K = 15 #(int(math.ceil( sum(l[i] * d[i] for i in range(n))/L ))) # valor limite nº de barras; menor limiar
print("Tamanho bobina jumbo:", L)
print("Demanda total:", sum(l[i] * d[i] for i in range(n)))
print("Valor limite K: ", K)
print("\n")

# Adição de variáveis
alpha = model.addVars(n, K, vtype=GRB.INTEGER, name="alpha", lb=0) # inteira não negativa Z+
y = model.addVars(K, vtype=GRB.BINARY, name="y")                   # binária

# Definição função objetivo: minimizar número de barras utilizadas
model.setObjective(
    sum(y[k] for k in range(K)),
    GRB.MINIMIZE
)

# Restrições
## Demanda
for j in range(n):
  model.addConstr(sum(alpha[j, k] for k in range(K)) >= d[j])

## Tamanho bobina jumbo
for k in range(K):
  model.addConstr(sum(l[i] * alpha[i, k] for i in range(n)) <= L * y[k])

## Restrição alpha >= 0 já add pelo uso de lb (lower bound) ao definir alpha
## Restrição y binário já add ao definir y

# Otimização
model.optimize()


# Escrita do modelo em formato LP
model.write('jumbo.lp')

# Salva a solução do problema
model.write('jumbo.sol')

# Exibe os resultados
if model.status == GRB.OPTIMAL:
    for i in range(n):
      for k in range(K):
        if alpha[i, k].X > 0:
          print(f"alpha[{i}, {k}] = {alpha[i, k].X}")

    for k in range(K):
      if y[k].X > 0:
        print(f"y[{k}] = {y[k].X}")

    print(f"z = {model.ObjVal}")

else:
    print("Solução ótima não encontrada.")

Tamanho bobina jumbo: 400.0
Demanda total: 4770.0
Valor limite K:  15


Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (linux64 - "Ubuntu 22.04.4 LTS")

CPU model: AMD EPYC 7B12, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 19 rows, 75 columns and 135 nonzeros
Model fingerprint: 0x98c69cb0
Variable types: 0 continuous, 75 integer (15 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 4e+01]
Presolve time: 0.00s
Presolved: 19 rows, 75 columns, 135 nonzeros
Variable types: 0 continuous, 75 integer (15 binary)
Found heuristic solution: objective 13.0000000

Root relaxation: objective 1.192500e+01, 43 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

  

In [35]:
# Padrões de corte encontrados (por barra)

print("\n" + "=" * 45)
print("Padrões de corte encontrados (por bobina):")
print("=" * 45)

## Cabeçalho com os comprimentos dos itens
header = "Bobina | " + " | ".join([f"{int(li):>4}" for li in l])
print(header)
print("-" * (len(header) + 3))

## Para cada bobina usada, mostrar a linha com os cortes
for k in range(K):
    if y[k].X > 0.5:
        linha = f"{k:>6} |"
        for i in range(n):
            qtd = int(alpha[i, k].X + 0.5)
            cell = f"{qtd:>4}" if qtd > 0 else "    "
            linha += f" {cell} |"
        print(linha)

print("=" * 45)
print(f"Total de bobinas utilizadas: {int(model.ObjVal)}")
print("=" * 45)


Padrões de corte encontrados (por bobina):
Bobina |   40 |   45 |   55 |   60
-------------------------------------
     0 |      |    4 |    4 |      |
     1 |      |      |    4 |    3 |
     2 |      |      |    4 |    3 |
     3 |      |      |    4 |    3 |
     4 |    7 |      |    2 |      |
     5 |    4 |      |    3 |    1 |
     6 |      |      |    4 |    3 |
     8 |      |    4 |    4 |      |
     9 |    1 |    8 |      |      |
    10 |      |      |    4 |    3 |
    12 |      |      |    5 |    2 |
    13 |      |    4 |    4 |      |
Total de bobinas utilizadas: 12


In [36]:
# Padrões de corte únicos encontrados, usos e perdas
from collections import defaultdict

# Contar padrões de corte e calcular perdas
padroes_unicos = defaultdict(int)
padroes_perda = {}

for k in range(K):
    if y[k].X > 0.5:
        padrao = tuple(int(alpha[i, k].X + 0.5) for i in range(n))
        padroes_unicos[padrao] += 1
        if padrao not in padroes_perda:
            usado = sum(padrao[i] * l[i] for i in range(n))
            perda = L - usado
            padroes_perda[padrao] = perda

print(f"Total de bobinas utilizadas: {int(model.ObjVal)}")
print(f"\nNúmero de padrões de corte distintos: {len(padroes_unicos)}")

# Exibir a tabela final
print("\n" + "=" * 45)
print(f"{'Padrão de corte':<20} | {'Usos':^5} | {'Perda':^6}")
print("-" * 45)

for padrao in padroes_unicos:
    usos = padroes_unicos[padrao]
    perda = padroes_perda[padrao]
    print(f"{str(padrao):<20} | {usos:^5} | {perda:^6.1f}")

print("=" * 45)

# Perda total
perda_total = sum(padroes_unicos[p] * padroes_perda[p] for p in padroes_unicos)
print(f"Perda total acumulada: {perda_total:.1f}")
print("=" * 45)


Total de bobinas utilizadas: 12

Número de padrões de corte distintos: 6

Padrão de corte      | Usos  | Perda 
---------------------------------------------
(0, 4, 4, 0)         |   3   |  0.0  
(0, 0, 4, 3)         |   5   |  0.0  
(7, 0, 2, 0)         |   1   |  10.0 
(4, 0, 3, 1)         |   1   |  15.0 
(1, 8, 0, 0)         |   1   |  0.0  
(0, 0, 5, 2)         |   1   |  5.0  
Perda total acumulada: 30.0
